<a href="https://colab.research.google.com/github/Penguinbeanie/Capstone-Project/blob/dev_branch/Capstone_Project_Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading Model

In [1]:
import google.generativeai as genai
from IPython.display import display, Markdown
from google.colab import userdata
import json

# retrieving the key stored in Colab
key = userdata.get('GOOGLE_API_KEY')

# configure the key for calling GenAI model
genai.configure(api_key=key)

# load model
model = genai.GenerativeModel("gemini-1.5-flash")

# Sorting Original Questionnaire Questions by Question Type

In [ ]:
import json
import requests
from google.colab import files

input_files = [
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire1.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire2.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire3.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire4.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire5.json"
]

# Dictionary to hold questions grouped by type
question_types = {}

# Iterate through each file
for file_url in input_files:
    response = requests.get(file_url)
    data = response.json()
    for entry in data:
        q_type = entry['type']
        if q_type not in question_types:
            question_types[q_type] = []
        question_types[q_type].append(entry)

# Save each question type to separate JSON files
output_files = []
for q_type, questions in question_types.items():
    filename = f"/content/{q_type}.json"
    with open(filename, 'w') as f:
        json.dump(questions, f, indent=4)
    output_files.append(filename)

print("Files have been created.")


Files have been created.


# JSON To String For Context

In [ ]:
# Load files

import json
import requests

file_path_date = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/type_seperated/DATE.json'
file_path_multi = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/type_seperated/MULTI_SELECT.json'
file_path_number = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/type_seperated/NUMBER.json'
file_path_single = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/type_seperated/SINGLE_SELECT.json'
file_path_text = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/type_seperated/TEXT.json'

# Function to load JSON from URL
def load_json_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
    return response.json()

# Load JSON data from URLs
try:
    json_data_date = load_json_from_url(file_path_date)
    json_data_multi = load_json_from_url(file_path_multi)
    json_data_number = load_json_from_url(file_path_number)
    json_data_single = load_json_from_url(file_path_single)
    json_data_text = load_json_from_url(file_path_text)

    # Convert JSON data to strings (if needed)
    json_string_date = json.dumps(json_data_date, indent=4)
    json_string_multi = json.dumps(json_data_multi, indent=4)
    json_string_number = json.dumps(json_data_number, indent=4)
    json_string_single = json.dumps(json_data_single, indent=4)
    json_string_text = json.dumps(json_data_text, indent=4)

except requests.exceptions.RequestException as e:
    print(f"Error loading JSON files: {e}")


# Running the Model

In [ ]:
# prompt

multiVar = 10
singleVar = 10
numberVar = 50
textVar = 50
dateVar = 50

prompt_multi = f"""
                Using the context below as a template, create {multiVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "MULTI-SELECT". The "options" array should contain multiple unique objects,
                each with the "option" key and a meaningful value representing an available choice. No IDs should be included.
                Generate multiple unique examples of questions that make sense in context for the type "MULTI-SELECT".
                Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Questionnnaire for evaluation. Diverse.

                Context:
                {json_string_multi}
                """

prompt_single = f"""
                Using the context below as a template, create {singleVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "SINGLE-SELECT". The "options" array should contain multiple unique objects,
                each with the "option" key and a meaningful value representing an available choice. No IDs should be included.
                Generate multiple unique examples of questions that make sense in context for the type "SINGLE-SELECT".
                Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Questionnnaire for evaluation. Communications.

                Context:
                {json_string_single}
                """

prompt_text = f"""
                Using the context below as a template, create {textVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "TEXT", the "options" array always contains exactly one object with
                the "option" set to "Text", and no IDs are included. Generate multiple unique examples of questions that make sense in context
                for the type "TEXT". They should be open ended. Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Questionnnaire for evaluation. Communications. (General topic in the field of communications, not directly the word communications)

                Context:
                {json_string_text}
                """

prompt_number = f"""
                Using the context below as a template, create {numberVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "NUMBER", the "options" array always contains exactly one object with
                the "option" set to the category the question best, and no IDs are included. Generate multiple unique examples of questions that make sense in context
                for the type "NUMBER". Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Questionnnaire for evaluation. Convention/Fair.

                Context:
                {json_string_number}
                """

prompt_date = f"""
                Using the context below as a template, create {dateVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "DATE", the "options" array always contains exactly one object with
                the "option" set to "Date", and no IDs are included. Generate multiple unique examples of questions that make sense in context
                for the type "DATE". Make sure the question specifically ask for a date. Add "Provide a date." at the end of each question.
                Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Questionnnaire for evaluation. Healthcare.

                Context:
                {json_string_date}
                """


# responses
responses = {
    #"prompt_multi": "Questionnaire_Multi_Artificial.json",
    #"prompt_single": "Questionnaire_Single_Artificial.json",
    #"prompt_date": "Questionnaire_Date_Artificial.json",
    #"prompt_number": "Questionnaire_Number_Artificial.json",
    "prompt_text": "Questionnaire_Text_Artificial.json"
}

for response_name, response_file in responses.items():
    prompt = globals()[response_name]  # Get the prompt string
    response = model.generate_content(prompt)  # Get model's response

    try:
        response_data = json.loads(response.text)  # Parse the response text as JSON
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for {response_name}: {e}")
        continue

    # Write the parsed JSON to a file
    with open(response_file, "w") as json_file:
        json.dump(response_data, json_file, indent=4)

    print(f"JSON file '{response_file}' has been created")

JSON file 'Questionnaire_Text_Artificial.json' has been created


#JSON to Dataframe

###MULTI

In [2]:
import json
import pandas as pd
import requests

#Load the JSON data
questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_Diverse.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_HeavyIndustry.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_Sales.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_SoftwareDev.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data
    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df_MULTI = pd.concat(dfs, ignore_index=True)

df_for_csv = df_MULTI.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('MULTI_combined.csv', index=False)

df_MULTI

,type,question,options
0,MULTI_SELECT,Which social media platforms do you use for ma...,"[Facebook, Instagram, Twitter, LinkedIn, TikTo..."
1,MULTI-SELECT,What types of marketing content do you find mo...,"[Blog posts, Videos, Infographics, Case studie..."
2,MULTI-SELECT,What are your primary goals for this marketing...,"[Increase brand awareness, Generate leads, Dri..."
3,MULTI-SELECT,What are your preferred methods of customer co...,"[Email, Phone, Chat, Social media, In-person m..."
4,MULTI-SELECT,What challenges are you facing in your current...,"[Lack of budget, Lack of time, Lack of resourc..."
...,...,...,...
186,MULTI-SELECT,What types of software development tools have ...,"[Integrated Development Environments (IDEs), S..."
187,MULTI-SELECT,What types of communication protocols are you ...,"[HTTP, HTTPS, TCP/IP, UDP, REST, GraphQL]"
188,MULTI-SELECT,What types of design patterns have you impleme...,"[Creational patterns, Structural patterns, Beh..."
189,MULTI-SELECT,What are your preferred methods for maintainin...,"[Code reviews, Static analysis, Unit testing, ..."


###SINGLE

In [ ]:
import json
import pandas as pd
import requests

#Load the JSON data

questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_ArtIndustry.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_Communications.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_Diverse.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_SoftwareDev.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data

    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df_SINGLE = pd.concat(dfs, ignore_index=True)

df_for_csv = df_SINGLE.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('SINGLE_combined.csv', index=False)

df_SINGLE


,type,question,options
0,SINGLE-SELECT,What is your preferred medium for artistic exp...,"[Painting, Sculpture, Digital Art, Photography..."
1,SINGLE-SELECT,What style of art are you most drawn to?,"[Abstract, Realism, Surrealism, Impressionism,..."
2,SINGLE-SELECT,What is your primary role in the art and desig...,"[Artist, Designer, Art Director, Graphic Desig..."
3,SINGLE-SELECT,What software do you primarily use for your work?,"[Adobe Photoshop, Adobe Illustrator, Adobe InD..."
4,SINGLE-SELECT,What is your preferred color palette?,"[Warm Colors, Cool Colors, Monochromatic, Anal..."
...,...,...,...
145,SINGLE-SELECT,What is your experience with data streaming te...,"[Beginner, Intermediate, Advanced]"
146,SINGLE-SELECT,What is your experience with big data technolo...,"[Beginner, Intermediate, Advanced]"
147,SINGLE-SELECT,What is your experience with artificial intell...,"[Beginner, Intermediate, Advanced]"
148,SINGLE-SELECT,What is your experience with blockchain techno...,"[Beginner, Intermediate, Advanced]"


### NUMBER

In [ ]:
import json
import pandas as pd
import requests

#Load the JSON data

questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/NUMBER/Questionnaire_Number_Artificial_BusinessAndJob.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/NUMBER/Questionnaire_Number_Artificial_HeavyIndustries.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/NUMBER/Questionnaire_Number_Artificial_Diverse.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/NUMBER/Questionnaire_Number_Artificial_Customer.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data

    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df_NUMBER = pd.concat(dfs, ignore_index=True)

df_for_csv = df_NUMBER.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('NUMBER_combined.csv', index=False)

df_NUMBER

,type,question,options
0,NUMBER,What is your employee ID number?,[employee ID]
1,NUMBER,What is the project budget in dollars?,[budget]
2,NUMBER,How many employees are in the department?,[employee count]
3,NUMBER,What is the annual revenue target?,[revenue]
4,NUMBER,What is the client account number?,[account number]
...,...,...,...
180,NUMBER,What is your pant size?,[pant size]
181,NUMBER,How many times have you traveled internationally?,[international travels]
182,NUMBER,How many credit cards do you have?,[credit card count]
183,NUMBER,What is your estimated net worth?,[net worth]


### DATE

In [6]:
import json
import pandas as pd
import requests

#Load the JSON data

questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/DATE/Questionnaire_Date_Artificial_Customer.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/DATE/Questionnaire_Date_Artificial_HeavyIndustries.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/DATE/Questionnaire_Date_Artificial_Technology.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/DATE/Questionnaire_Date_Artificial_Healthcare.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data

    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df_DATE = pd.concat(dfs, ignore_index=True)

df_for_csv = df_DATE.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('DATE_combined.csv', index=False)

df_DATE

,type,question,options
0,DATE,What is your date of birth? Provide a date.,[Date]
1,DATE,When did you purchase our product? Provide a d...,[Date]
2,DATE,What date did the issue begin? Provide a date.,[Date]
3,DATE,When is your next appointment? Provide a date.,[Date]
4,DATE,What is the delivery date? Provide a date.,[Date]
...,...,...,...
183,DATE,When did you last consume alcohol? Provide a d...,[Date]
184,DATE,When was your last physical therapy appointmen...,[Date]
185,DATE,When did you last have a significant injury? P...,[Date]
186,DATE,When did you start experiencing shortness of b...,[Date]


### TEXT

In [4]:
import json
import pandas as pd
import requests

#Load the JSON data

questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/TEXT/Questionnaire_Text_Artificial_Communications.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/TEXT/Questionnaire_Text_Artificial_HeavyIndustries.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/TEXT/Questionnaire_Text_Artificial_CustomerSupport.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/TEXT/Questionnaire_Text_Artificial_Convention.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data

    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df_TEXT = pd.concat(dfs, ignore_index=True)

df_for_csv = df_TEXT.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('TEXT_combined.csv', index=False)

df_TEXT

,type,question,options
0,TEXT,Describe your experience.,[Text]
1,TEXT,What were the key takeaways?,[Text]
2,TEXT,Explain your approach.,[Text]
3,TEXT,Summarize the main points.,[Text]
4,TEXT,What challenges did you encounter?,[Text]
...,...,...,...
173,TEXT,Describe any areas where the convention fell s...,[Text]
174,TEXT,What aspects of the convention fostered a sens...,[Text]
175,TEXT,How effectively did the convention balance com...,[Text]
176,TEXT,What were the strengths and weaknesses of the ...,[Text]


# Answer Generation

### MULTI

In [21]:
import time
from google.api_core import exceptions

df_MULTI_with_answers = df_MULTI.copy().head(2)

# Generate a single compact prompt for all variations
def generate_minimal_prompt(row):
    return f"""Question: {row['question']}
            Options: {', '.join(row['options'])}

            Provide 5 different responses to this multiple choice question. Each response should select from the given options. Chose a different
            cobination of options (or single option) for each response. Each answer (V, C, Q, E, A) is to be given by a seperate person.
            Each label (V_Sel, C_Sel, Q_Sel, E_Sel, A_Sel) should contain the options corresponding to the response.
            Format exactly as follows:
            V: [verbose response, not exceeding 3 sentences]
            V_Sel: [options corresponding to the response V]
            C: [concise response]
            C_Sel: [options corresponding to the response C]
            Q: [colloquial response, no 'honestly']
            Q_Sel: [options corresponding to the response Q]
            E: [explanatory response, not exceeding 3 sentences]
            E_Sel: [options corresponding to the response E]
            A: [mildly annoyed response, no 'ugh']
            A_Sel: [options corresponding to the response A]"""

# Define the prefixes for each response type
prefixes = {
    'verbose': 'V:',
    'verbose_Gemini_label': 'V_Sel: ',
    'concise': 'C:',
    'concise_Gemini_label': 'C_Sel: ',
    'colloquial': 'Q:',
    'colloquial_Gemini_label': 'Q_Sel: ',
    'explanatory': 'E:',
    'explanatory_Gemini_label': 'E_Sel: ',
    'annoyed': 'A:',
    'annoyed_Gemini_label': 'A_Sel: '
}

# Parse the response text into separate variations
def parse_responses(text):
    answers = {}
    lines = text.split('\n')

    # Extract each response type
    for key, prefix in prefixes.items():
        try:
            # Find the line starting with this prefix
            response_line = next((line for line in lines if line.strip().startswith(prefix)), '')
            # Remove the prefix and trim
            response = response_line.replace(prefix, '', 1).strip()
            answers[f"answer_{key}"] = response if response else f"Error: No {key} response found"
        except Exception as e:
            answers[f"answer_{key}"] = f"Error parsing {key} response: {str(e)}"

    return answers

# Process a single row with retries
def process_row(row):
    prompt = generate_minimal_prompt(row)
    for attempt in range(3):
        try:
            response = model.generate_content(prompt)
            answers = parse_responses(response.text)
            time.sleep(3)
            return answers
        except exceptions.TooManyRequests:
            if attempt < 2:
                time.sleep(2 ** attempt)
            else:
                return {f"answer_{key}": "Error: Rate limit exceeded."
                       for key in prefixes}
        except Exception as e:
            print(f"Generated text was:\n{response.text}\n")  # Print the response text
            print(f"Error: {str(e)}")
            return {f"answer_{key}": f"Error: {str(e)}"
                  for key in prefixes}

# Process the dataframe
answer_columns = df_MULTI_with_answers.apply(process_row, axis=1)

# Add columns to dataframe
for key in prefixes:
    df_MULTI_with_answers[f"answer_{key}"] = answer_columns.apply(lambda x: x[f"answer_{key}"])

# Save to CSV
df_for_csv_MULTI_answer = df_MULTI_with_answers.copy()
df_for_csv_MULTI_answer['options'] = df_for_csv_MULTI_answer['options'].apply(lambda x: '; '.join(x))
df_for_csv_MULTI_answer.to_csv('MULTI_answer_combined.csv', index=False, sep='>')

df_MULTI_with_answers

,type,question,options,answer_verbose,answer_verbose_Gemini_label,answer_concise,answer_concise_Gemini_label,answer_colloquial,answer_colloquial_Gemini_label,answer_explanatory,answer_explanatory_Gemini_label,answer_annoyed,answer_annoyed_Gemini_label
0,MULTI_SELECT,Which social media platforms do you use for ma...,"[Facebook, Instagram, Twitter, LinkedIn, TikTo...",We utilize Facebook and Instagram primarily fo...,"Facebook, Instagram, YouTube","Facebook, Instagram, TikTok.","Facebook, Instagram, TikTok","We're all over Facebook and Insta, plus we're...","Facebook, Instagram, TikTok","For visual content, we rely on Instagram and Y...","Facebook, Instagram, YouTube, LinkedIn","We use Facebook and Twitter, mostly. It's a ...","Facebook, Twitter"
1,MULTI-SELECT,What types of marketing content do you find mo...,"[Blog posts, Videos, Infographics, Case studie...",I find that a combination of video content and...,"Videos, Blog posts, Case studies",Videos and infographics are most effective.,"Videos, Infographics","Gotta say, videos are king, but a good email n...","Videos, Email newsletters",Email newsletters are effective for consistent...,"Email newsletters, Infographics, Videos","It depends on the audience and the product, bu...","Case studies, Blog posts"


### SINGLE

In [ ]:
import time
from google.api_core import exceptions

df_SINGLE_with_answers = df_SINGLE.copy().head(2)

def generate_minimal_prompt(row):
    return f"""Question: {row['question']}
            Options: {', '.join(row['options'])}
            Provide 5 different responses to this single choice question. Each response should select from the given options. Chose a different
            option for each response (if enough options exist for all 5 responses, otherwise repeat options after having already used all of them).
            Each answer (V, C, Q, E, A) is to be given by a seperate person.
            Each label (V_Sel, C_Sel, Q_Sel, E_Sel, A_Sel) should contain the option corresponding to the response. Format exactly as follows:
            V: [verbose response, not exceeding 3 sentences]
            V_Sel: [option corresponding to the response V]
            C: [concise response]
            C_Sel: [option corresponding to the response C]
            Q: [colloquial response, no 'honestly']
            Q_Sel: [option corresponding to the response Q]
            E: [explanatory response, not exceeding 3 sentences]
            E_Sel: [option corresponding to the response E]
            A: [mildly annoyed response, no 'ugh']
            A_Sel: [option corresponding to the response A]"""

# Define the prefixes for each response type
prefixes = {
    'verbose': 'V:',
    'verbose_Gemini_label': 'V_Sel: ',
    'concise': 'C:',
    'concise_Gemini_label': 'C_Sel: ',
    'colloquial': 'Q:',
    'colloquial_Gemini_label': 'Q_Sel: ',
    'explanatory': 'E:',
    'explanatory_Gemini_label': 'E_Sel: ',
    'annoyed': 'A:',
    'annoyed_Gemini_label': 'A_Sel: '
}

def parse_responses(text):
    answers = {}
    lines = text.split('\n')
    for key, prefix in prefixes.items():
        try:
            for line in lines:
                if line.startswith(prefix):
                    line = line.removeprefix(prefix)
                    answers[f"answer_{key}"] = line if line else f"Error: No {key} response found"
        except Exception as e:
                answers[f"answer_{key}"] = f"Error parsing {key} response: {str(e)}"

    return answers

def process_row(row):
    prompt = generate_minimal_prompt(row)
    for attempt in range(3):
        try:
            response = model.generate_content(prompt)
            answers = parse_responses(response.text)
            time.sleep(3)
            return answers
        except exceptions.TooManyRequests:
            if attempt < 2:
                time.sleep(2 ** attempt)
            else:
                return {f"answer_{key}": "Error: Rate limit exceeded."
                       for key in prefixes}
        except Exception as e:
            print(f"Generated text was:\n{response.text}\n")  # Print the response text
            print(f"Error: {str(e)}")
            return {f"answer_{key}": f"Error: {str(e)}"
                  for key in prefixes}

answer_columns = df_SINGLE_with_answers.apply(process_row, axis=1)

# Add columns to dataframe
for key in prefixes:
    df_SINGLE_with_answers[f"answer_{key}"] = answer_columns.apply(lambda x: x[f"answer_{key}"])

df_for_csv_SINGLE_answer = df_SINGLE_with_answers.copy()
df_for_csv_SINGLE_answer['options'] = df_for_csv_SINGLE_answer['options'].apply(lambda x: '; '.join(x))
df_for_csv_SINGLE_answer.to_csv('SINGLE_answer_combined.csv', index=False, sep='>')

df_SINGLE_with_answers

,type,question,options,answer_verbose,answer_verbose_Gemini_label,answer_concise,answer_concise_Gemini_label,answer_colloquial,answer_colloquial_Gemini_label,answer_explanatory,answer_explanatory_Gemini_label,answer_annoyed,answer_annoyed_Gemini_label
0,SINGLE-SELECT,What is your preferred medium for artistic exp...,"[Painting, Sculpture, Digital Art, Photography...",I find the fluidity and vibrancy of painting ...,Painting,Sculpture.,Sculpture,"Digital art's the way to go, man. So much you...",Digital Art,I prefer photography because it allows me to ...,Photography,"Fine, if I *must* choose, I suppose graphic ...",Graphic Design
1,SINGLE-SELECT,What style of art are you most drawn to?,"[Abstract, Realism, Surrealism, Impressionism,...",I find myself most captivated by the dreamlik...,Surrealism,Abstract art.,Abstract,"Gotta go with Impressionism, man. Those blu...",Impressionism,Realism appeals to me the most because of its...,Realism,"Fine, if I have to pick, I suppose Pop Art i...",Pop Art


### NUMBER

In [ ]:
import time
from google.api_core import exceptions

df_NUMBER_with_answers = df_NUMBER.copy().head(2)

def generate_minimal_prompt(row):
    return f"""Question: {row['question']}
            Options: {row['options']}
            Provide 5 different responses to this single choice question, which asks for a number. The question provides a single option defining the type of number to be chosen.
            Each response should adhere to the given type but provide a unique answer. Some answers should be written as numbers, some such that the numbers are spelled out.
            Each answer (V, C, Q, E, A) is to be given by a separate person.
            Each label (V_Sel, C_Sel, Q_Sel, E_Sel, A_Sel) should contain just the number (with correct units) corresponding to the response. Format exactly as follows:
            V: [verbose response, not exceeding 3 sentences]
            V_Sel: [number corresponding to the response V]
            C: [concise response]
            C_Sel: [number corresponding to the response C]
            Q: [colloquial response, no 'honestly']
            Q_Sel: [number corresponding to the response Q]
            E: [explanatory response, not exceeding 3 sentences]
            E_Sel: [number corresponding to the response E]
            A: [mildly annoyed response, no 'ugh']
            A_Sel: [number corresponding to the response A]"""

# Define the prefixes for each response type
prefixes = {
    'verbose': 'V:',
    'verbose_Gemini_label': 'V_Sel: ',
    'concise': 'C:',
    'concise_Gemini_label': 'C_Sel: ',
    'colloquial': 'Q:',
    'colloquial_Gemini_label': 'Q_Sel: ',
    'explanatory': 'E:',
    'explanatory_Gemini_label': 'E_Sel: ',
    'annoyed': 'A:',
    'annoyed_Gemini_label': 'A_Sel: '
}

def parse_responses(text):
    answers = {}
    lines = text.split('\n')
    for key, prefix in prefixes.items():
        try:
            for line in lines:
                if line.startswith(prefix):
                    line = line.removeprefix(prefix)
                    answers[f"answer_{key}"] = line if line else f"Error: No {key} response found"
        except Exception as e:
                answers[f"answer_{key}"] = f"Error parsing {key} response: {str(e)}"

    return answers

def process_row(row):
    prompt = generate_minimal_prompt(row)
    for attempt in range(3):
        try:
            response = model.generate_content(prompt)
            answers = parse_responses(response.text)
            time.sleep(3)
            return answers
        except exceptions.TooManyRequests:
            if attempt < 2:
                time.sleep(2 ** attempt)
            else:
                return {f"answer_{key}": "Error: Rate limit exceeded."
                       for key in prefixes}
        except Exception as e:
            print(f"Generated text was:\n{response.text}\n")  # Print the response text
            print(f"Error: {str(e)}")
            return {f"answer_{key}": f"Error: {str(e)}"
                  for key in prefixes}

answer_columns = df_NUMBER_with_answers.apply(process_row, axis=1)

for key in prefixes:
    df_NUMBER_with_answers[f"answer_{key}"] = answer_columns.apply(lambda x: x[f"answer_{key}"])

df_for_csv_NUMBER_answer = df_NUMBER_with_answers.copy()
df_for_csv_NUMBER_answer['options'] = df_for_csv_NUMBER_answer['options'].apply(lambda x: '; '.join(x))
df_for_csv_NUMBER_answer.to_csv('NUMBER_answer_combined.csv', index=False, sep='>')

df_NUMBER_with_answers

,type,question,options,answer_verbose,answer_verbose_Gemini_label,answer_concise,answer_concise_Gemini_label,answer_colloquial,answer_colloquial_Gemini_label,answer_explanatory,answer_explanatory_Gemini_label,answer_annoyed,answer_annoyed_Gemini_label
0,NUMBER,What is your employee ID number?,[employee ID],My employee ID number is one hundred and twen...,123456,789012,789012,"My ID? It's 345678, dude.",345678,"My employee identification number, as assigne...",901234,The number you requested is 567890. I hope ...,567890
1,NUMBER,What is the project budget in dollars?,[budget],The project budget is allocated at five milli...,"$5,000,000","$12,750,000","$12,750,000","The budget? It's, like, two hundred and fif...","$250,000","The total project budget is $8,300,000. This...","$8,300,000",The budget is three million dollars. I've al...,"$3,000,000"


### DATE

In [19]:
import time
from google.api_core import exceptions

df_DATE_with_answers = df_DATE.copy().head(1)

def generate_minimal_prompt(row):
    return f"""Question: {row['question']}
            Options: {row['options']}
            Provide 5 different responses to this single choice question, which asks for a date. The question provides one option defining the type to be chosen.
            Each response should adhere to the given type but provide a unique answer. Some answers should be written as numbers, some such that the numbers are spelled out. Use different date formats.
            Assume quistion takers are from Europe.
            Each answer (V, C, Q, E, A) is to be given by a separate person.
            Each label (V_Sel, C_Sel, Q_Sel, E_Sel, A_Sel) should contain just the date corresponding to the response. Use this date format: dd-mm-yyyy and nothing else for the label.
            Format exactly as follows:
            V: [verbose response, not exceeding 3 sentences]
            V_Sel: [date corresponding to the response V]
            C: [concise response]
            C_Sel: [date corresponding to the response C]
            Q: [colloquial response, no 'honestly']
            Q_Sel: [date corresponding to the response Q]
            E: [explanatory response, not exceeding 3 sentences]
            E_Sel: [date corresponding to the response E]
            A: [mildly annoyed response, no 'ugh']
            A_Sel: [date corresponding to the response A]"""

# Define the prefixes for each response type
prefixes = {
    'verbose': 'V:',
    'verbose_Gemini_label': 'V_Sel: ',
    'concise': 'C:',
    'concise_Gemini_label': 'C_Sel: ',
    'colloquial': 'Q:',
    'colloquial_Gemini_label': 'Q_Sel: ',
    'explanatory': 'E:',
    'explanatory_Gemini_label': 'E_Sel: ',
    'annoyed': 'A:',
    'annoyed_Gemini_label': 'A_Sel: '
}

def parse_responses(text):
    answers = {}
    lines = text.split('\n')
    for key, prefix in prefixes.items():
        try:
            for line in lines:
                if line.startswith(prefix):
                    line = line.removeprefix(prefix)
                    answers[f"answer_{key}"] = line if line else f"Error: No {key} response found"
        except Exception as e:
                answers[f"answer_{key}"] = f"Error parsing {key} response: {str(e)}"

    return answers

def process_row(row):
    prompt = generate_minimal_prompt(row)
    for attempt in range(3):
        try:
            response = model.generate_content(prompt)
            answers = parse_responses(response.text)
            time.sleep(3)
            return answers
        except exceptions.TooManyRequests:
            if attempt < 2:
                time.sleep(2 ** attempt)
            else:
                return {f"answer_{key}": "Error: Rate limit exceeded."
                       for key in prefixes}
        except Exception as e:
            print(f"Generated text was:\n{response.text}\n")  # Print the response text
            print(f"Error: {str(e)}")
            return {f"answer_{key}": f"Error: {str(e)}"
                  for key in prefixes}

answer_columns = df_DATE_with_answers.apply(process_row, axis=1)

for key in prefixes:
    df_DATE_with_answers[f"answer_{key}"] = answer_columns.apply(lambda x: x[f"answer_{key}"])

df_for_csv_DATE_answer = df_DATE_with_answers.copy()
df_for_csv_DATE_answer['options'] = df_for_csv_DATE_answer['options'].apply(lambda x: '; '.join(x))
df_for_csv_DATE_answer.to_csv('DATE_answer_combined.csv', index=False, sep='>')

df_DATE_with_answers

,type,question,options,answer_verbose,answer_verbose_Gemini_label,answer_concise,answer_concise_Gemini_label,answer_colloquial,answer_colloquial_Gemini_label,answer_explanatory,answer_explanatory_Gemini_label,answer_annoyed,answer_annoyed_Gemini_label
0,DATE,What is your date of birth? Provide a date.,[Date],"My date of birth is the twenty-fifth of July,...",25-07-1987,03/04/1992,03-04-1992,"It's the eleventh of November, nineteen nine...",11-11-1995,"My birthday falls on the 14th of February, 20...",14-02-2001,"It's the first of January, two thousand. Wh...",01-01-2000


### TEXT

In [22]:
import time
from google.api_core import exceptions

df_TEXT_with_answers = df_TEXT.copy().head(4)

def generate_minimal_prompt(row):
    return f"""Question: {row['question']}
            Options: {row['options']}
            Provide 5 different responses to this single choice question, which asks for a text. The question provides a single option defining the type of answer to be chosen.
            Each response should adhere to the given type but provide a unique answer. The questions are open ended, therfore your answer should replicate a human's answer to such a question.
            Each answer (V, C, Q, E, A) is to be given by a separate person.
            Each label (V_Sel, C_Sel, Q_Sel, E_Sel, A_Sel) should contain the shortest possible topic summery.
            Format exactly as follows:
            V: [verbose response, not exceeding 3 sentences]
            V_Sel: [summery corresponding to the response V]
            C: [concise response]
            C_Sel: [summery corresponding to the response C]
            Q: [colloquial response, no 'honestly']
            Q_Sel: [summery corresponding to the response Q]
            E: [explanatory response, not exceeding 3 sentences]
            E_Sel: [summery corresponding to the response E]
            A: [mildly annoyed response, no 'ugh']
            A_Sel: [summery corresponding to the response A]"""

# Define the prefixes for each response type
prefixes = {
    'verbose': 'V:',
    'verbose_Gemini_label': 'V_Sel: ',
    'concise': 'C:',
    'concise_Gemini_label': 'C_Sel: ',
    'colloquial': 'Q:',
    'colloquial_Gemini_label': 'Q_Sel: ',
    'explanatory': 'E:',
    'explanatory_Gemini_label': 'E_Sel: ',
    'annoyed': 'A:',
    'annoyed_Gemini_label': 'A_Sel: '
}

def parse_responses(text):
    answers = {}
    lines = text.split('\n')
    for key, prefix in prefixes.items():
        try:
            for line in lines:
                if line.startswith(prefix):
                    line = line.removeprefix(prefix)
                    answers[f"answer_{key}"] = line if line else f"Error: No {key} response found"
        except Exception as e:
                answers[f"answer_{key}"] = f"Error parsing {key} response: {str(e)}"

    return answers

def process_row(row):
    prompt = generate_minimal_prompt(row)
    for attempt in range(3):
        try:
            response = model.generate_content(prompt)
            answers = parse_responses(response.text)
            time.sleep(3)
            return answers
        except exceptions.TooManyRequests:
            if attempt < 2:
                time.sleep(2 ** attempt)
            else:
                return {f"answer_{key}": "Error: Rate limit exceeded."
                       for key in prefixes}
        except Exception as e:
            print(f"Generated text was:\n{response.text}\n")  # Print the response text
            print(f"Error: {str(e)}")
            return {f"answer_{key}": f"Error: {str(e)}"
                  for key in prefixes}

answer_columns = df_TEXT_with_answers.apply(process_row, axis=1)

for key in prefixes:
    df_TEXT_with_answers[f"answer_{key}"] = answer_columns.apply(lambda x: x[f"answer_{key}"])

df_for_csv_TEXT_answer = df_TEXT_with_answers.copy()
df_for_csv_TEXT_answer['options'] = df_for_csv_TEXT_answer['options'].apply(lambda x: '; '.join(x))
df_for_csv_TEXT_answer.to_csv('TEXT_answer_combined.csv', index=False, sep='>')

df_TEXT_with_answers

,type,question,options,answer_verbose,answer_verbose_Gemini_label,answer_concise,answer_concise_Gemini_label,answer_colloquial,answer_colloquial_Gemini_label,answer_explanatory,answer_explanatory_Gemini_label,answer_annoyed,answer_annoyed_Gemini_label
0,TEXT,Describe your experience.,[Text],My experience has been a multifaceted journey...,Rich and rewarding journey,A continuous learning process.,Constant learning,"It's been a wild ride, man! Ups and downs, ...",Life's ups and downs,My experience is characterized by a gradual a...,Gradual skill & knowledge growth,"Well, it's been a mixed bag, to put it mildl...","Mixed experiences, needs clarification"
1,TEXT,What were the key takeaways?,[Text],The key takeaways were the significant increa...,"Customer satisfaction, software integration, t...","Increased customer satisfaction, software int...",Key findings summary,"So, like, the big things were happier custom...",Improved morale and customer satisfaction; sof...,The primary conclusions drawn were a positiv...,"Training success, software problems, communica...",The main points? Higher customer satisfactio...,"Training success, software problems, positive ..."
2,TEXT,Explain your approach.,[Text],My approach involved carefully considering al...,Methodical & thorough approach,Systemic analysis and response generation.,Systematic approach,"I just, like, thought about it for a sec and...",Intuitive response,I systematically analyzed the question to und...,Systematic response selection,"Look, I read the question and answered it. W...",Direct & succinct answer
3,TEXT,Summarize the main points.,[Text],The main points are the key takeaways or cent...,Key takeaways and central arguments,The core ideas.,Core ideas,"So, like, the main stuff, right? The import...",Important details,The main points are the most significant asp...,Significant aspects and overall message,Just tell me the important parts already. I...,Important parts of the text


# Notes

In [ ]:
'''
import pandas as pd
from transformers import pipeline

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Example DataFrame
data = {
    "text": [
        "I prefer to eat mostly bread.",
        "I love grilled salmon.",
        "Steak is my favorite dish.",
        "I enjoy cheeseburgers."
    ],
    "labels": [["Bread", "Fish", "Meat", "Burger"],  # Labels for each row
               ["Fish", "Meat", "Bread", "Burger"],
               ["Meat", "Burger", "Fish", "Bread"],
               ["Burger", "Meat", "Fish", "Bread"]]
}

df_test = pd.DataFrame(data)

df_test


# Function to classify each row
def classify_row(row):
    result = classifier(row["text"], candidate_labels=row["labels"])
    return result["labels"][0], result["scores"][0]


# Apply the classification to each row

df["predicted_labels"], df["scores"] = zip(*df.apply(lambda row: classify_row(row), axis=1))

# Display the DataFrame
print(df)
'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


'\n# Function to classify each row\ndef classify_row(row):\n    result = classifier(row["text"], candidate_labels=row["labels"])\n    return result["labels"][0], result["scores"][0]\n\n\n# Apply the classification to each row\n\ndf["predicted_labels"], df["scores"] = zip(*df.apply(lambda row: classify_row(row), axis=1))\n\n# Display the DataFrame\nprint(df)\n'

In [ ]:
'''
import pandas as pd

data_test = {
    "text": [
        "I prefer to eat mostly bread.",
        "I love grilled salmon.",
        "Steak is my favorite dish.",
        "I enjoy cheeseburgers."
    ],
    "labels": [["Bread", "Fish", "Meat", "Burger"],  # Labels for each row
               ["Fish", "Meat", "Bread", "Burger"],
               ["Meat", "Burger", "Fish", "Bread"],
               ["Burger", "Meat", "Fish", "Bread"]]
}

df_test = pd.DataFrame(data)

df_test

print(df)
'''

'\nimport pandas as pd \n\ndata_test = {\n    "text": [\n        "I prefer to eat mostly bread.",\n        "I love grilled salmon.",\n        "Steak is my favorite dish.",\n        "I enjoy cheeseburgers."\n    ],\n    "labels": [["Bread", "Fish", "Meat", "Burger"],  # Labels for each row\n               ["Fish", "Meat", "Bread", "Burger"],\n               ["Meat", "Burger", "Fish", "Bread"],\n               ["Burger", "Meat", "Fish", "Bread"]]\n}\n\ndf_test = pd.DataFrame(data)\n\ndf_test\n\nprint(df)\n'